Preparing Data

<p> We will be combining data from three files </p>
<ol>
<li> We will select name, artist, genre, and number of grammy awards from grammySongs_1999-2019.csv. This information is not immediately available from the sheet, so we will group by artist and name and count the number of grammys each song has won. </li>
<li> We will select artist, name, peak position, average position, and weeks on chart from billboardHot100_1999-2019. </li>
<li> We will select artist, name, energy, danceability, tempo, and valence from data/songAtrributes_1999-2019.csv. </li>
</ol>

In [23]:
import pandas as pd
import numpy as np

In [2]:
grammyAwards = pd.read_csv("data/grammySongs_1999-2019.csv")
# grammyAwards = grammyAwards[['Name', 'Artist', 'Genre', 'GrammyAward']]
grammyAwards = grammyAwards[['Name', 'Artist', 'GrammyAward']]
grammyAwards = grammyAwards.groupby(['Name', 'Artist']).count().reset_index()
grammyAwards = grammyAwards.applymap(lambda s: s.upper() if type(s) == str else s)
print(grammyAwards.shape)
grammyAwards.head(5)

(366, 3)


,Name,Artist,GrammyAward
0,'TIL SUMMER COMES AROUND,KEITH URBAN,1
1,24 FRAMES,"ASON ISBELL, SONGWRITER",1
2,24K MAGIC,BRUNO MARS,1
3,500 MILES HIGH,"CHICK COREA, SOLOIST",1
4,99 PROBLEMS,JAY-Z,1


In [3]:
billboardHot100 = pd.read_csv('data/billboardHot100_1999-2019.csv')
billboardHot100 = billboardHot100[['Artists', 'Name', 'Peak.position', 'Weeks.on.chart']]
billboardHot100['Weeks.on.chart'] = billboardHot100['Weeks.on.chart'].fillna(1)
billboardHot100['Peak.position'] = billboardHot100['Peak.position'].fillna(100)
# billboardHot100.groupby(by=['Artist', 'Name']).min('Peak.position').max('Weeks.on.chart')
billboardHot100 = billboardHot100.groupby(['Artists', 'Name']).agg({'Peak.position':'min', 'Weeks.on.chart':'max'})[['Peak.position','Weeks.on.chart']].reset_index()
billboardHot100 = billboardHot100.applymap(lambda s: s.upper() if type(s) == str else s)
billboardHot100.head(5)

,Artists,Name,Peak.position,Weeks.on.chart
0,"""WEIRD AL"" YANKOVIC",CANADIAN IDIOT,82.0,3.0
1,"""WEIRD AL"" YANKOVIC",WORD CRIMES,39.0,2.0
2,'N SYNC,BYE BYE BYE,4.0,23.0
3,'N SYNC,GONE,11.0,24.0
4,'N SYNC,IT'S GONNA BE ME,1.0,25.0


In [4]:
songAttributes = pd.read_csv('data/songAttributes_1999-2019.csv')
songAttributes = songAttributes[['Name', 'Artist', 'Energy', 'Danceability', 'Tempo', 'Valence']]
songAttributes = songAttributes.applymap(lambda s: s.upper() if type(s) == str else s)
songAttributes.head(5)

,Name,Artist,Energy,Danceability,Tempo,Valence
0,WELCOME ALL AGAIN,COLLECTIVE SOUL,0.904,0.520,106.022,0.365
1,FUZZY,COLLECTIVE SOUL,0.709,0.581,120.027,0.408
2,DIG,COLLECTIVE SOUL,0.918,0.572,144.061,0.370
3,YOU,COLLECTIVE SOUL,0.661,0.596,111.975,0.183
4,MY DAYS,COLLECTIVE SOUL,0.808,0.520,92.721,0.666


In [5]:
songAttributes2 = pd.read_csv('data/songAttributesV2.csv')
# songAttributes.columns
songAttributes2 = songAttributes2[['song_title', 'artist', 'energy', 'danceability', 'tempo', 'valence']]
songAttributes2.rename(
    columns={"song_title":"Name", "artist":"Artist"}, inplace=True)
songAttributes2 = songAttributes2.applymap(lambda s: s.upper() if type(s) == str else s)
songAttributes2.head(5)

,Name,Artist,energy,danceability,tempo,valence
0,MASK OFF,FUTURE,0.434,0.833,150.062,0.286
1,REDBONE,CHILDISH GAMBINO,0.359,0.743,160.083,0.588
2,XANNY FAMILY,FUTURE,0.412,0.838,75.044,0.173
3,MASTER OF NONE,BEACH HOUSE,0.338,0.494,86.468,0.230
4,PARALLEL LINES,JUNIOR BOYS,0.561,0.678,174.004,0.904


In [6]:
# grammyAwards.set_index(['Name', 'Artist']).join(songAttributes.set_index(['Name', 'Artist']), how='left').reset_index()
# inProgress = grammyAwards.merge(songAttributes, on=['Name', 'Artist'], how='left').drop_duplicates(subset=['Name'], keep='first')
inProgress = grammyAwards.merge(songAttributes, on=['Name', 'Artist'], how='left').drop_duplicates(subset=['Name', 'Artist'], keep='first')
# inProgress
inProgress.merge(songAttributes2, on=['Name', 'Artist'], how='left')
# grammyAwards.drop_duplicates(subset=['Name'], keep='first')

,Name,Artist,GrammyAward,Energy,Danceability,Tempo,Valence,energy,danceability,tempo,valence
0,'TIL SUMMER COMES AROUND,KEITH URBAN,1,0.629,0.570,127.907,0.308,NaN,NaN,NaN,NaN
1,24 FRAMES,"ASON ISBELL, SONGWRITER",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,24K MAGIC,BRUNO MARS,1,0.803,0.818,106.970,0.632,NaN,NaN,NaN,NaN
3,500 MILES HIGH,"CHICK COREA, SOLOIST",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,99 PROBLEMS,JAY-Z,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
361,YOU DON'T KNOW MY NAME,ALICIA KEYS,1,0.663,0.263,167.271,0.250,NaN,NaN,NaN,NaN
362,YOU GOT ME,THE ROOTS FEATURING ERYKAH BADU,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
363,YOU SAY,"LAUREN DAIGLE; LAUREN DAIGLE, JASON INGRAM & P...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364,YOU WANT IT DARKER,LEONARD COHEN,1,0.327,0.674,108.995,0.494,NaN,NaN,NaN,NaN


In [36]:
# a = np.array([])
# a = np.append(a, grammyAwards.Artist.unique())
grammyAwards.Artist.unique()
# a = np.append(a, billboardHot100.Artists.unique())
# np.savetxt('myfile.csv', a, delimiter=',', fmt="%s")
# len(a)
# grammyAwards.Artist.unique()

array(['KEITH URBAN', 'ASON ISBELL, SONGWRITER', 'BRUNO MARS',
       'CHICK COREA, SOLOIST', 'JAY-Z', 'MIGUEL', 'LENNY KRAVITZ',
       'CHRISTINA AGUILERA', 'PARAMORE',
       'ENDRICK DUCKWORTH, KAWAN PRATHER, MARK ANTHONY SPEARS & PHARRELL WILLIAMS, SONGWRITERS',
       'KENDRICK LAMAR', 'LALAH HATHAWAY', 'NEIL YOUNG',
       'SYSTEM OF A DOWN', 'LADY GAGA', 'SKRILLEX\xa0FEATURING SIRAH',
       'THE CIVIL WARS', 'WILLI.AM', 'MARY J BLIGE',
       'BONO, ADAM CLAYTON, EDGE, LARRY MULLEN JR), SONGWRITER (U2', 'U2',
       'CARRIE UNDERWOOD', 'SLIPKNOT', 'CHER',
       'HE.R. FEATURING DANIEL CAESAR', 'LITTLE BIG TOWN', 'FANTASIA',
       'DAVID BOWIE', 'JAMIE FOXX & T-PAIN', 'RASCAL FLATTS',
       'TONY BENNETT & AMY WINEHOUSE', 'ELLA MAI', 'GREEN DAY', 'STING',
       'FAITH HILL', 'EVANESCENCE FEATURING PAUL MCCOY',
       'CHRIS STAPLETON', 'ANDERSON PAAK', 'KACEY MUSGRAVES', 'PRINCE',
       'JUSTIN TIMBERLAKE, ANNA KENDRICK, GWEN STEFANI, JAMES CORDEN, ZOOEY DESCHANEL, WALT DO